In [ ]:
# Import
import pandas as pd
import nimblephysics as nimble
import numpy as np
import os

from COMETH import Skeleton, DynamicSkeleton
from sf_utils import input_sim, rotation, sk_model, comp_filter, inverse_kinematics
from sf_utils.metrics import body_pose_metrics

In [ ]:
#--------File names-------------#
imu_data_path = os.path.abspath('raw_imu_data/imu_s1_acting1.csv')
vicon_data_path = os.path.abspath('./totalcapture/vicon')
subj = 's1'
action = 'acting1'
saving_dir = f'./{subj}_{action}'

#----------Other constants--------#
# Framerate
dt = 1/60

# Camera rotation matrix respect to thorx IMU
theta_cam = np.deg2rad(-75)
R_cam_imu = np.array([[np.cos(theta_cam), 0, np.sin(theta_cam)],[0,1,0],[-np.sin(theta_cam),0,np.cos(theta_cam)]])

# Ratio between camera and imu framerates
ratio = 10

R_correction = np.array([np.eye(3), np.eye(3), np.eye(3)])

In [ ]:
# Compute ground-truth
target, pos_gt, offset = sk_model.compute_gt(os.path.join(subj, action), vicon_data_path)

In [ ]:
# Getting accelerometer and gyroscope data from file
imu_cont = input_sim.ImuData(imu_data_path)
imu_cont.read_imu_csv()
imu = imu_cont.acc_read
acc, gyro = imu_cont.get_imu_array(['right_elbow', 'left_elbow', 'sternum'])

#Creating human model for ground-truth data
s_gt = sk_model.create_body_model(os.path.join(subj, action), vicon_path = vicon_data_path, body_node_list=['humerus_r', 'humerus_l', "thorax"])

# Creating human model for pose estimation
s = sk_model.create_body_model(os.path.join(subj, action), vicon_path = vicon_data_path, body_node_list=['humerus_r', 'humerus_l', "thorax"])
s._nimble.setVelocities(np.zeros((49,1)))
s._nimble.setAccelerations(np.zeros((49,1)))
start_pos = s._nimble.getPositions()


# Initilizing orient estimator (one for each IMU)
cf0 = comp_filter.ComplementaryFilter()
cf1 = comp_filter.ComplementaryFilter()
cf2 = comp_filter.ComplementaryFilter()

# Initializing metric manager
mm = body_pose_metrics.MetricManager(s=s, target=target, offset=offset)

# Setting IMU starting orientation in the pose estimation model
T=np.array([np.eye(4), np.eye(4), np.eye(4)])
T[:,:3,:3]= rotation.batch_quat_to_rotmat(imu_cont.get_world_orient(['right_elbow', 'left_elbow', 'sternum'])[:,0])
s.setImuWorldTransform(T)

for i in range(0, imu['n_frames']):
    # setting gt position in gt model
    s_gt._nimble.setPositions(pos_gt[i])

    pos_target = None
    valid_pos = False

    # At camera framerate, check if any keypoint visible
    if (i%ratio)==0:
        # getting camera gt world position and orientation
        camera_pos = s_gt.getImuWorldPosition()[2]
        R_cam_w = s_gt.getImuWorldOrient()[2,:3,:3]@R_cam_imu

        # getting gt kaypoints positions
        pos_target = (s_gt._nimble.getJointWorldPositions(s_gt.joints)).reshape(-1,3)
        
        # calculate which keypoint is visible
        pos_target = input_sim.camera_simulation(pos_target, camera_pos=camera_pos, R_cam= R_cam_w)
        
        # check if both hands is visible
        if pos_target is not None:
            if not (np.all(np.isnan(pos_target[[6,4]]))):
                valid_pos = True

    # updating orient estimators with new data and calculate gravity vector
    r_0, p_0, y_0 = cf0.update(accel=(R_correction[0]@acc[0,i].T).reshape((3)), gyro=gyro[0,i])
    g_0 = rotation.euler_to_gravity(r_0, p_0, y_0).reshape(-1)
    
    r_1, p_1, y_1 = cf1.update(accel=(R_correction[1]@acc[1,i].T).reshape((3)), gyro=gyro[1,i])
    g_1 = rotation.euler_to_gravity(r_1, p_1, y_1).reshape(-1)
    
    r_2, p_2, y_2 = cf2.update(accel=(R_correction[2]@acc[2,i].T).reshape((3)), gyro=gyro[2,i])
    g_2 = rotation.euler_to_gravity(r_2, p_2, y_2).reshape(-1)
    
    gravity = np.array([g_0,g_1,g_2]).reshape(-1)


    no_acc = np.all(np.abs((np.linalg.norm(acc[:,i], axis=-1)-1))>0.5)
    if (pos_target is not None) or (i==0) or (no_acc):
        gravity = None
    
    inverse_kinematics.estimate_position_from_IMUs_qp(s,acc_target=gravity, gyro_target=None,position_target=pos_target)
    

    # Correcting the orient estimator if the conditions are satisfy
    if i==0 or (pos_target is not None):
        T = np.array([np.eye(4),np.eye(4),np.eye(4)])
        R_v_world = s.getImuWorldOrient().reshape(-1,4,4)[:,:3,:3]
        T[:,:3,:3]= R_v_world

        if valid_pos or i==0 or no_acc:
            cf0.correct(T[0].reshape(4,4))
            cf1.correct(T[1].reshape(4,4))
            cf2.correct(T[2].reshape(4,4))

            
    # Saving this frame pose estimation
    q = s._nimble.getPositions().copy()
    q[3:6]=start_pos[3:6]
    
    s._nimble.setPositions(q)
    
    mm.append_new_pos(q)

In [ ]:
mm.save_metrics(dir = saving_dir)